# <strong><font color=#A52A2A></span> MIR_HW5 </strong>

98101566 -> Mohammadreza Daviran
<br>
98105919 -> Reza Erfan Arani
<br>
98106434 -> Mohammad Khodadadi Aski
<br>

<strong><font color=#B22222></span> Please run this notebook on google colab or on a computer with higher than 12GB memory RAM! </strong>

In this notebook, we fully implement the second scenario.

In this implementation the paragraphs Namnak's texts were considered as nodes. We connect two sentences(or paragraphs) If they have more than a certain limit duplicate words. Finally, by using link analysis algorithms, we find the most important (or central) sentences of the given texts.


# <strong><font color=#5F9EA0></span> Download the files </strong> 

In [ ]:
import requests
import json

In [ ]:
!git clone https://github.com/language-ml/course-nlp-ir-1-text-exploring.git

Cloning into 'course-nlp-ir-1-text-exploring'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 401 (delta 2), reused 0 (delta 0), pack-reused 397
Receiving objects: 100% (401/401), 98.78 MiB | 21.87 MiB/s, done.
Resolving deltas: 100% (197/197), done.
Checking out files: 100% (245/245), done.


In [ ]:
%cd course-nlp-ir-1-text-exploring/exploring-datasets/health

/content/course-nlp-ir-1-text-exploring/exploring-datasets/health


In [ ]:
!ls

hidoctor-1.json  hidoctor-4.json  namnak-2.json  namnak-5.json
hidoctor-2.json  hidoctor-5.json  namnak-3.json  README.md
hidoctor-3.json  namnak-1.json	  namnak-4.json


Our dataset is json so first of all we merge all the files to 'bio.json' file.

In [ ]:
def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
        with open(f1, 'r') as infile:
            result.extend(json.load(infile))

    with open('bio.json', 'w') as output_file:
        json.dump(result, output_file)

# files = ['hidoctor-1.json', 'hidoctor-2.json', 'hidoctor-3.json','hidoctor-4.json','hidoctor-5.json', 'namnak-1.json', 'namnak-2.json', 'namnak-3.json','namnak-4.json','namnak-5.json']
files = ['namnak-1.json', 'namnak-2.json', 'namnak-3.json','namnak-4.json','namnak-5.json']
merge_JsonFiles(files)

By loading the json file, we extract all the paragraphs and save them in *documents* for future work.

In [ ]:
f = open('bio.json')
bioset = json.load(f)
f.close()

In [ ]:
nutrition_documents = []
new_in_med_documents = []
dental_documents = []
family_health_documents = []
mental_health_documents = []
hair_skin_documents = []
prophylaxis_disease_documents = []

for i in bioset:
  if (i['categories'][1] == 'تغذیه'):
    nutrition_documents.append(i['paragraphs'])
  elif (i['categories'][1] == 'تازه های سلامت'):
    new_in_med_documents.append(i['paragraphs'])
  elif (i['categories'][1] == 'دهان و دندان'):
    dental_documents.append(i['paragraphs'])
  elif (i['categories'][1] == 'سلامت خانواده'):
    family_health_documents.append(i['paragraphs'])
  elif (i['categories'][1] == 'سلامت روان'):
    mental_health_documents.append(i['paragraphs'])
  elif (i['categories'][1] == 'پوست و مو'):
    hair_skin_documents.append(i['paragraphs'])
  elif (i['categories'][1] == 'پیشگیری و بیماریها'):
    prophylaxis_disease_documents.append(i['paragraphs'])

In [ ]:
x = set()
for i in bioset:
  x.add(i['categories'][1])
NUM_OF_CATEGORIES = len(x)
x

{'تازه های سلامت',
 'تغذیه',
 'دهان و دندان',
 'سلامت خانواده',
 'سلامت روان',
 'پوست و مو',
 'پیشگیری و بیماریها'}

# <strong><font color=#5F9EA0></span> Preprocessing </strong> 

First we import the requirements for preprocessing the data.

In [ ]:
import random 
import tqdm
import nltk
from nltk.corpus import brown
from nltk import word_tokenize
import csv
import re
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import random 
import nltk
import codecs

In [ ]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 5.1 MB/s 
     |████████████████████████████████| 233 kB 55.9 MB/s 
     |████████████████████████████████| 1.4 MB 50.2 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=c441887fb098e1c906df8523ac1682eca29479a1e45b35ac5b3276a267e02e65
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154122 sha256=db87e5c4efc23aaa56b231a96950757afabda58a66378772c88dbe51075d35ea
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [ ]:
from __future__ import unicode_literals
from hazm import *

**normalize**

normalizer function normilizes the docs we extracted and saves them 

In [ ]:
normalizer = Normalizer()

nutrition_doc_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(nutrition_documents)]
new_in_med_doc_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(new_in_med_documents)]
dental_doc_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(dental_documents)]
family_health_doc_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(family_health_documents)]
mental_health_doc_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(mental_health_documents)]
hair_skin_doc_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(hair_skin_documents)]
prophylaxis_disease_doc_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(prophylaxis_disease_documents)]

100%|██████████| 125/125 [00:00<00:00, 246.25it/s]


In [ ]:
nutrition_doc_normalized_edited = []
for doc in nutrition_doc_normalized:
  temp = []
  for x in doc:
    temp.append(x.replace('\u200c', ' '))
  nutrition_doc_normalized_edited.append(temp)

new_in_med_doc_normalized_edited = []
for doc in new_in_med_doc_normalized:
  temp = []
  for x in doc:
    temp.append(x.replace('\u200c', ' '))
  new_in_med_doc_normalized_edited.append(temp)

dental_doc_normalized_edited = []
for doc in dental_doc_normalized:
  temp = []
  for x in doc:
    temp.append(x.replace('\u200c', ' '))
  dental_doc_normalized_edited.append(temp)

family_health_doc_normalized_edited = []
for doc in family_health_doc_normalized:
  temp = []
  for x in doc:
    temp.append(x.replace('\u200c', ' '))
  family_health_doc_normalized_edited.append(temp)

mental_health_doc_normalized_edited = []
for doc in mental_health_doc_normalized:
  temp = []
  for x in doc:
    temp.append(x.replace('\u200c', ' '))
  mental_health_doc_normalized_edited.append(temp)

hair_skin_doc_normalized_edited = []
for doc in hair_skin_doc_normalized:
  temp = []
  for x in doc:
    temp.append(x.replace('\u200c', ' '))
  hair_skin_doc_normalized_edited.append(temp)

prophylaxis_disease_doc_normalized_edited = []
for doc in prophylaxis_disease_doc_normalized:
  temp = []
  for x in doc:
    temp.append(x.replace('\u200c', ' '))
  prophylaxis_disease_doc_normalized_edited.append(temp)

**wording**

here we tokenize the sentences in each normilized document and then we save them in the doc_sentences array 

In [ ]:
nutrition_doc_sentences = [sent_tokenize(' '.join(x)) for x in tqdm.tqdm(nutrition_doc_normalized_edited)]
new_in_med_doc_sentences = [sent_tokenize(' '.join(x)) for x in tqdm.tqdm(new_in_med_doc_normalized_edited)]
dental_doc_sentences = [sent_tokenize(' '.join(x)) for x in tqdm.tqdm(dental_doc_normalized_edited)]
family_health_doc_sentences = [sent_tokenize(' '.join(x)) for x in tqdm.tqdm(family_health_doc_normalized_edited)]
mental_health_doc_sentences = [sent_tokenize(' '.join(x)) for x in tqdm.tqdm(mental_health_doc_normalized_edited)]
hair_skin_doc_sentences = [sent_tokenize(' '.join(x)) for x in tqdm.tqdm(hair_skin_doc_normalized_edited)]
prophylaxis_disease_doc_sentences = [sent_tokenize(' '.join(x)) for x in tqdm.tqdm(prophylaxis_disease_doc_normalized_edited)]

100%|██████████| 125/125 [00:00<00:00, 3045.42it/s]


In [ ]:
all_nutrition_doc_sentences = []
for paragraph in nutrition_doc_sentences:
  for sentence in paragraph:
    sent = [sentence]
    all_nutrition_doc_sentences.append(sent)

all_new_in_med_doc_sentences = []
for paragraph in new_in_med_doc_sentences:
  for sentence in paragraph:
    sent = [sentence]
    all_new_in_med_doc_sentences.append(sent)

all_dental_doc_sentences = []
for paragraph in dental_doc_sentences:
  for sentence in paragraph:
    sent = [sentence]
    all_dental_doc_sentences.append(sent)

all_family_health_doc_sentences = []
for paragraph in family_health_doc_sentences:
  for sentence in paragraph:
    sent = [sentence]
    all_family_health_doc_sentences.append(sent)

all_mental_health_doc_sentences = []
for paragraph in mental_health_doc_sentences:
  for sentence in paragraph:
    sent = [sentence]
    all_mental_health_doc_sentences.append(sent)

all_hair_skin_doc_sentences = []
for paragraph in hair_skin_doc_sentences:
  for sentence in paragraph:
    sent = [sentence]
    all_hair_skin_doc_sentences.append(sent)

all_prophylaxis_disease_doc_sentences = []
for paragraph in prophylaxis_disease_doc_sentences:
  for sentence in paragraph:
    sent = [sentence]
    all_prophylaxis_disease_doc_sentences.append(sent)

**tokenize**

we tokenize the words in the doc_sentences array and the we save them into doc_tokens array by tqdm tools.

In [ ]:
nutrition_doc_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(all_nutrition_doc_sentences)]
new_in_med_doc_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(all_new_in_med_doc_sentences)]
dental_doc_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(all_dental_doc_sentences)]
family_health_doc_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(all_family_health_doc_sentences)]
mental_health_doc_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(all_mental_health_doc_sentences)]
hair_skin_doc_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(all_hair_skin_doc_sentences)]
prophylaxis_disease_doc_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(all_prophylaxis_disease_doc_sentences)]

100%|██████████| 6855/6855 [00:00<00:00, 28277.26it/s]


Then by using itertools save them in all_tokens. Because we want to exclude the stopwords from our tokens.

In [ ]:
all_nutrition_tokens = []
for i in nutrition_doc_tokens:
  all_nutrition_tokens.append(list(itertools.chain.from_iterable(i)))

all_new_in_med_tokens = []
for i in new_in_med_doc_tokens:
  all_new_in_med_tokens.append(list(itertools.chain.from_iterable(i)))

all_dental_tokens = []
for i in dental_doc_tokens:
  all_dental_tokens.append(list(itertools.chain.from_iterable(i)))

all_family_health_tokens = []
for i in family_health_doc_tokens:
  all_family_health_tokens.append(list(itertools.chain.from_iterable(i)))

all_mental_health_tokens = []
for i in mental_health_doc_tokens:
  all_mental_health_tokens.append(list(itertools.chain.from_iterable(i)))

all_hair_skin_tokens = []
for i in hair_skin_doc_tokens:
  all_hair_skin_tokens.append(list(itertools.chain.from_iterable(i)))

all_prophylaxis_disease_tokens = []
for i in prophylaxis_disease_doc_tokens:
  all_prophylaxis_disease_tokens.append(list(itertools.chain.from_iterable(i)))

In [ ]:
# import json
# import numpy as np

# lists = np.array(all_nutrition_tokens).tolist()
# nutrition_json_str = json.dumps(lists)

# lists = np.array(all_new_in_med_tokens).tolist()
# new_in_med_json_str = json.dumps(lists)

# lists = np.array(all_dental_tokens).tolist()
# dental_json_str = json.dumps(lists)

# lists = np.array(all_family_health_tokens).tolist()
# family_health_json_str = json.dumps(lists)

# lists = np.array(all_mental_health_tokens).tolist()
# mental_health_json_str = json.dumps(lists)

# lists = np.array(all_hair_skin_tokens).tolist()
# hair_skin_json_str = json.dumps(lists)

# lists = np.array(all_prophylaxis_disease_tokens).tolist()
# prophylaxis_disease_json_str = json.dumps(lists)

**<font color='#DD4343'>Note:**
  
  If you have weak computer **do not** run this section!
  <br>
  it will take almost an hour to just delete the stop words.
  <br>
  This also applies for lemmatization...
  <br>
  <br>
  If you have chosen to delete stop words and lemmatize documents:

  *   Change 'all_nutrition_tokens' with 'all_nutrition_tokens_nonstop'
    - nutrition can be replaced by 'dental', 'mental_health', ...

**</font>**

**Delete stop words**

here we 


1.   download the persian stopwords from bellow link
2.   then we delete them from all_tokens arraylist cause they have no value for us
3. save the tokens which are not stopwords in all_tokens_nonstop arraylist




In [ ]:
import codecs
link = "https://raw.githubusercontent.com/sobhe/hazm/master/hazm/data/stopwords.dat"
temp_file = open('stopwords.txt', 'wb').write(requests.get(link, allow_redirects=True).content)

In [ ]:
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]

In [ ]:
all_nutrition_tokens_nonstop = []
for doc in all_nutrition_tokens:
  all_nutrition_tokens_nonstop.append([t for t in tqdm.tqdm(doc) if t not in stopwords])
  
all_dental_tokens_nonstop = []
for doc in all_dental_tokens:
  all_dental_tokens_nonstop.append([t for t in tqdm.tqdm(doc) if t not in stopwords])

all_new_in_med_tokens_nonstop = []
for doc in all_new_in_med_tokens:
  all_new_in_med_tokens_nonstop.append([t for t in tqdm.tqdm(doc) if t not in stopwords])

all_family_health_tokens_nonstop = []
for doc in all_family_health_tokens:
  all_family_health_tokens_nonstop.append([t for t in tqdm.tqdm(doc) if t not in stopwords])

all_mental_health_tokens_nonstop = []
for doc in all_mental_health_tokens:
  all_mental_health_tokens_nonstop.append([t for t in tqdm.tqdm(doc) if t not in stopwords])

all_hair_skin_tokens_nonstop = []
for doc in all_hair_skin_tokens:
  all_hair_skin_tokens_nonstop.append([t for t in tqdm.tqdm(doc) if t not in stopwords])

all_prophylaxis_disease_tokens_nonstop = []
for doc in all_prophylaxis_disease_tokens:
  all_prophylaxis_disease_tokens_nonstop.append([t for t in tqdm.tqdm(doc) if t not in stopwords])


**lemmatization, stemming**

here we lemmatize and stem the tokens 

In [ ]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_lemma_set(tok, opt=1):
    if opt == 1:
        return stemmer.stem(tok)
    if opt == 2:
        return lemmatizer.lemmatize(tok)

In [ ]:
opt = 2

all_nutrition_tokens_nonstop_lemstem = []
all_dental_tokens_nonstop_lemstem = []
all_new_in_med_tokens_nonstop_lemstem = []
all_family_health_tokens_nonstop_lemstem = []
all_mental_health_tokens_nonstop_lemstem = []
all_hair_skin_tokens_nonstop_lemstem = []
all_prophylaxis_disease_tokens_nonstop_lemstem = []

for doc in all_dental_tokens_nonstop:
  all_nutrition_tokens_nonstop_lemstem.append([get_lemma_set(t, opt) for t in tqdm.tqdm(doc)])

for doc in all_nutrition_tokens_nonstop:
  all_dental_tokens_nonstop_lemstem.append([get_lemma_set(t, opt) for t in tqdm.tqdm(doc)])

for doc in all_new_in_med_tokens_nonstop:
  all_new_in_med_tokens_nonstop_lemstem.append([get_lemma_set(t, opt) for t in tqdm.tqdm(doc)])

for doc in all_family_health_tokens_nonstop:
  all_family_health_tokens_nonstop_lemstem.append([get_lemma_set(t, opt) for t in tqdm.tqdm(doc)])

for doc in all_mental_health_tokens_nonstop:
  all_mental_health_tokens_nonstop_lemstem.append([get_lemma_set(t, opt) for t in tqdm.tqdm(doc)])

for doc in all_hair_skin_tokens_nonstop:
  all_hair_skin_tokens_nonstop_lemstem.append([get_lemma_set(t, opt) for t in tqdm.tqdm(doc)])

for doc in all_prophylaxis_disease_tokens_nonstop:
  all_prophylaxis_disease_tokens_nonstop_lemstem.append([get_lemma_set(t, opt) for t in tqdm.tqdm(doc)])

In [ ]:
symbols = [
           '!',
           '?',
           '؟',
           '!',
           ',',
           '،',
           '.',
           '.',
           '[',
           ']',
           '(',
           ')',
           '{',
           '}',
           '؛',
           ';',
           '"',
           '/',
           '\\',
]

In [ ]:
all_nutrition_tokens_nonsymbol = []
for doc in all_nutrition_tokens:
  doc_tokens_nonsymbol = []
  for word in doc:
    if word not in symbols:
      doc_tokens_nonsymbol.append(word)
  all_nutrition_tokens_nonsymbol.append(doc_tokens_nonsymbol)

all_new_in_med_tokens_nonsymbol = []
for doc in all_new_in_med_tokens:
  doc_tokens_nonsymbol = []
  for word in doc:
    if word not in symbols:
      doc_tokens_nonsymbol.append(word)
  all_new_in_med_tokens_nonsymbol.append(doc_tokens_nonsymbol)

all_dental_tokens_nonsymbol = []
for doc in all_dental_tokens:
  doc_tokens_nonsymbol = []
  for word in doc:
    if word not in symbols:
      doc_tokens_nonsymbol.append(word)
  all_dental_tokens_nonsymbol.append(doc_tokens_nonsymbol)

all_family_health_tokens_nonsymbol = []
for doc in all_family_health_tokens:
  doc_tokens_nonsymbol = []
  for word in doc:
    if word not in symbols:
      doc_tokens_nonsymbol.append(word)
  all_family_health_tokens_nonsymbol.append(doc_tokens_nonsymbol)

all_mental_health_tokens_nonsymbol = []
for doc in all_mental_health_tokens:
  doc_tokens_nonsymbol = []
  for word in doc:
    if word not in symbols:
      doc_tokens_nonsymbol.append(word)
  all_mental_health_tokens_nonsymbol.append(doc_tokens_nonsymbol)

all_hair_skin_tokens_nonsymbol = []
for doc in all_hair_skin_tokens:
  doc_tokens_nonsymbol = []
  for word in doc:
    if word not in symbols:
      doc_tokens_nonsymbol.append(word)
  all_hair_skin_tokens_nonsymbol.append(doc_tokens_nonsymbol)

all_prophylaxis_disease_tokens_nonsymbol = []
for doc in all_prophylaxis_disease_tokens:
  doc_tokens_nonsymbol = []
  for word in doc:
    if word not in symbols:
      doc_tokens_nonsymbol.append(word)
  all_prophylaxis_disease_tokens_nonsymbol.append(doc_tokens_nonsymbol)

In [ ]:
import json
import numpy as np

lists = np.array(all_nutrition_tokens_nonsymbol).tolist()
nutrition_json_str = json.dumps(lists)

lists = np.array(all_new_in_med_tokens_nonsymbol).tolist()
new_in_med_json_str = json.dumps(lists)

lists = np.array(all_dental_tokens_nonsymbol).tolist()
dental_json_str = json.dumps(lists)

lists = np.array(all_family_health_tokens_nonsymbol).tolist()
family_health_json_str = json.dumps(lists)

lists = np.array(all_mental_health_tokens_nonsymbol).tolist()
mental_health_json_str = json.dumps(lists)

lists = np.array(all_hair_skin_tokens_nonsymbol).tolist()
hair_skin_json_str = json.dumps(lists)

lists = np.array(all_prophylaxis_disease_tokens_nonsymbol).tolist()
prophylaxis_disease_json_str = json.dumps(lists)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specif

In [ ]:
with open('all_nutrition_tokens_nonsymbol.json', 'w') as o:
    json.dump(nutrition_json_str, o)

with open('all_new_in_med_tokens_nonsymbol.json', 'w') as o:
    json.dump(nutrition_json_str, o)

with open('all_dental_tokens_nonsymbol.json', 'w') as o:
    json.dump(nutrition_json_str, o)

with open('all_family_health_tokens_nonsymbol.json', 'w') as o:
    json.dump(nutrition_json_str, o)

with open('all_mental_health_tokens_nonsymbol.json', 'w') as o:
    json.dump(nutrition_json_str, o)

with open('all_hair_skin_tokens_nonsymbol.json', 'w') as o:
    json.dump(nutrition_json_str, o)

with open('all_prophylaxis_disease_tokens_nonsymbol.json', 'w') as o:
    json.dump(nutrition_json_str, o)

  

# <strong><font color=#5F9EA0></span> tf-idf </strong> 

In this section we build our Vectorizer!

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
vectorizer = TfidfVectorizer(use_idf = True, norm ='l2', ngram_range=(1,2), stop_words=None)

# <strong><font color=#5F9EA0></span> Link Analysis </strong> 

We have 7 different Categories in our documents. Each category will get a seperate link analysis below.

### <strong><font color=#CF9E50></span> Nutrition Docs Analysis </strong>

First Category is: **<font color="#A54565"> Nutrition </font>**
<br>
First we fit and transform our vectorizer with our sentences.

In [ ]:
vectorizer.fit([' '.join(doc) for doc in all_nutrition_tokens_nonsymbol])

TfidfVectorizer(ngram_range=(1, 2))

In [ ]:
X = vectorizer.transform([' '.join(doc) for doc in all_nutrition_tokens_nonsymbol])

Now we build the adjacency matrix.

In [ ]:
P = X.dot(X.T)

In [ ]:
from sklearn.preprocessing import normalize

We normalize the matrix with L1 norm.

In [ ]:
P_norm = normalize(P, norm='l1')

Checking that sum of a line is actualy equal to 1.

*You can change '2' to any number you want within the range*

In [ ]:
import numpy as np

np.sum(P_norm[2,:])

1.0000000000000027

In [ ]:
import networkx as nx

Now we want a graph from numpy matrix.
The numpy matrix is interpreted as an adjacency matrix for the graph.

In [ ]:
G = nx.from_numpy_matrix(P_norm.toarray())

Building page rank...

In [ ]:
page_rank = nx.pagerank(G, alpha=0.9)

Hubs and Authorities...

In [ ]:
h, a = nx.hits(G)

Printing the most 5(changable) important sentences. 

This has been done both for page_rank and hits/authorities. 

You can see that they are almost alike.

In [ ]:
# all_nutrition_doc_sentences[np.argmax(list(page_rank.values()))]
argmaxes = np.argpartition(list(page_rank.values()), -5)[-5:]
for i in argmaxes:
  print(all_nutrition_doc_sentences[i], end='\n-----\n')

['ویتامین E - توکوترینولز: ویتامین E یک جز مهم در بسیاری از محصولات مراقبت از پوست است، اما بیشتر ویتامین E مصنوعی است و واقعا هیچ کار خوبی انجام نمی دهد با این حال، ویتامین E در محصولات طبیعی برای پوست و موی شما مفید است، این نوع ویتامین E به عنوان a شناخته می شود و یک آنتی اکسیدان قوی است.']
-----
['قطعا می دانید که مصرف ویتامین، راه پیشگیری از بیماری های مختلف می باشد که سیر نیز یکی از همین ویتامین ها است که سلامت شما و از بین بردن سموم بدن در طولانی مدت را تضمین می کند.']
-----
['روغن سبوس برنج نیز غنی از ماده ای به نام ارزیانول است، که یک مبارز رادیکال آزاد با اثرات آنتی اکسیدانی قوی تر نسبت به ویتامین E است و منحصر به فرد است؛ علاوه بر استفاده از روغن سبوس برنج بر روی پوست و مو، شما می توانید آن را نیز بخورید و این روغن قادر به درمان بسیاری از مزایای التیام و تقویت پوست خواهد بود.، همچنین آنطور که در بخش سلامت نمناک خواهیم گفت می توانید از آن در سس سالاد استفاده کنید چون به راحتی با ترکیبات دیگر بدون جدا کردن ترکیب می شود.']
-----
['روغن سبوس برنج پوست چرب را بهبود می دهد: با این

In [ ]:
argmaxes = np.argpartition(list(h.values()), -5)[-5:]
for i in argmaxes:
  print(all_nutrition_doc_sentences[i], end='\n-----\n')

['روغن سبوس برنج پوست چرب را بهبود می دهد: با این که روغن سبوس برنج نیز یک مرطوب کننده بسیار غنی است، اما در پاسخ به سوال برخی از شما کاربران از بخش سلامت نمناک باید بگوییم که این روغن پوست را چرب نمی کند.']
-----
['ترمیم بافت بدن فسفر ماده معدنی است که به شکل گیری ساختار سلول های بدن کمک می کند، به ساخت ماده زمینه ای استخوان کمک می کند و مولکول هایی که منبع خوبی از انرژی برای بدن ما هستند را تقویت می کند.']
-----
['قطعا می دانید که مصرف ویتامین، راه پیشگیری از بیماری های مختلف می باشد که سیر نیز یکی از همین ویتامین ها است که سلامت شما و از بین بردن سموم بدن در طولانی مدت را تضمین می کند.']
-----
['روغن سبوس برنج نیز غنی از ماده ای به نام ارزیانول است، که یک مبارز رادیکال آزاد با اثرات آنتی اکسیدانی قوی تر نسبت به ویتامین E است و منحصر به فرد است؛ علاوه بر استفاده از روغن سبوس برنج بر روی پوست و مو، شما می توانید آن را نیز بخورید و این روغن قادر به درمان بسیاری از مزایای التیام و تقویت پوست خواهد بود.، همچنین آنطور که در بخش سلامت نمناک خواهیم گفت می توانید از آن در سس سالاد استفاده کن

In [ ]:
argmaxes = np.argpartition(list(a.values()), -5)[-5:]
for i in argmaxes:
  print(all_nutrition_doc_sentences[i], end='\n-----\n')

['روغن سبوس برنج پوست چرب را بهبود می دهد: با این که روغن سبوس برنج نیز یک مرطوب کننده بسیار غنی است، اما در پاسخ به سوال برخی از شما کاربران از بخش سلامت نمناک باید بگوییم که این روغن پوست را چرب نمی کند.']
-----
['ترمیم بافت بدن فسفر ماده معدنی است که به شکل گیری ساختار سلول های بدن کمک می کند، به ساخت ماده زمینه ای استخوان کمک می کند و مولکول هایی که منبع خوبی از انرژی برای بدن ما هستند را تقویت می کند.']
-----
['قطعا می دانید که مصرف ویتامین، راه پیشگیری از بیماری های مختلف می باشد که سیر نیز یکی از همین ویتامین ها است که سلامت شما و از بین بردن سموم بدن در طولانی مدت را تضمین می کند.']
-----
['روغن سبوس برنج نیز غنی از ماده ای به نام ارزیانول است، که یک مبارز رادیکال آزاد با اثرات آنتی اکسیدانی قوی تر نسبت به ویتامین E است و منحصر به فرد است؛ علاوه بر استفاده از روغن سبوس برنج بر روی پوست و مو، شما می توانید آن را نیز بخورید و این روغن قادر به درمان بسیاری از مزایای التیام و تقویت پوست خواهد بود.، همچنین آنطور که در بخش سلامت نمناک خواهیم گفت می توانید از آن در سس سالاد استفاده کن

We do the same thing for other 6 categories. 

(if you want you can uncomment hits/authorities part to see their results too)

### <strong><font color=#CF9E50></span> News in Med Docs Analysis </strong>

In [ ]:
vectorizer.fit([' '.join(doc) for doc in all_new_in_med_tokens_nonsymbol])

TfidfVectorizer(ngram_range=(1, 2))

In [ ]:
X = vectorizer.transform([' '.join(doc) for doc in all_new_in_med_tokens_nonsymbol])

In [ ]:
P = X.dot(X.T)

In [ ]:
P_norm = normalize(P, norm='l1')

In [ ]:
np.sum(P_norm[2,:])

1.0000000000000009

In [ ]:
G = nx.from_numpy_matrix(P_norm.toarray())

In [ ]:
page_rank = nx.pagerank(G, alpha=0.9)

In [ ]:
h, a = nx.hits(G)

In [ ]:
argmaxes = np.argpartition(list(page_rank.values()), -5)[-5:]
for i in argmaxes:
  print(all_new_in_med_doc_sentences[i], end='\n-----\n')

['راز هایی در مورد ویتامین ها و مکمل ها که پزشکان به دوستان خود می گویند به تازگی متخصصان ایده های جدیدی را درباره مصرف ویتامین ها ارائه داده اند و با استناد به آن ها متوجه می شویم که توصیه های آنان در مورد ویتامین نسبت به همیشه فرق می کند که حال در این قسمت از سلامت نمناک برخی از این کشفیات پزشکان را در مورد مصرف ویتامین را برای شما فاش خواهیم کرد.']
-----
['پس قبل از هرکاری سعی کنید که استفاده از فیبر را از غذاهای طبیعی داشته باشید اما اگر باز هم کمبود را احساس می کردید، می توانید از مکمل های ۳ گرمی فیبر یک بار در روز استفاده کنید و بعد از سه روز می توانید استفاده از این مکمل ها را افزایش دهید اما باید وضعیت سلامتی و بیماری هایی مثل دیابت را نزد پزشک مشخص کنید که فیبر باعث کاهش قند خون و کاهش جذب داروها می شود.']
-----
['این امر توسط دانشمندان به عنوان یک اقدام بسیار خطرناک برای ساختار سلامتی انسان معرفی شده است زیرا به واسطه نگاه کردن به اشعه ساتع از نمایشگر در اتاق تاریک و پیش از خواب در حقیقت به مغز خود آسیبی را وارد می کنید که منجر به عدم خواب راحت در طول شب خواهد شد و این امر به

In [ ]:
# argmaxes = np.argpartition(list(h.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')

# argmaxes = np.argpartition(list(a.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')

### <strong><font color=#CF9E50></span> Dental Docs Analysis </strong>

In [ ]:
vectorizer.fit([' '.join(doc) for doc in all_dental_tokens_nonsymbol])
X = vectorizer.transform([' '.join(doc) for doc in all_dental_tokens_nonsymbol])

In [ ]:
P = X.dot(X.T)

In [ ]:
P_norm = normalize(P, norm='l1')
np.sum(P_norm[2,:])

0.9999999999999989

In [ ]:
G = nx.from_numpy_matrix(P_norm.toarray())

In [ ]:
page_rank = nx.pagerank(G, alpha=0.9)
h, a = nx.hits(G)

In [ ]:
argmaxes = np.argpartition(list(page_rank.values()), -5)[-5:]
for i in argmaxes:
  print(all_dental_doc_sentences[i], end='\n-----\n')

['رعایت نکردن بهداشت دندانی: و استفاده نامناسب از مسواک می تواند باعث تیره شدن رنگ دندان شود و علت آن این است که باکتری ها در این قسمت تجمع پیدا می کنند و باعث ایجاد حفره و تغییر رنگ در آن ها می شود و یکی از خطرناک ترین نتایج چنین حفره هایی این است که فرد ممکن است ناحیه ای از دندان را به کل از دست بدهد.']
-----
['کشیدن دندان متاسفانه، کشیدن دندان های زیاد باعث از بین رفتن استخوان فک می شود، استخوان فک نیاز به مراقب دارد، اگر مراقبت از آن انجام نشود، سیستم ایمنی او را به عنوان یک عامل خطر شناسایی می کند و شروع به شکستن استخوان می کند، این اتفاق باعث می شود که دندان های بیشتری از بین برود، اگر دندانتان را در اثر تصادف از دست داده اید، این موضوع می تواند بر سلامت فرد تاثیر بگذارد، علاوه بر آن، اگر دندان آسیاب کشیده شوند، حفره های سینوسی بزرگ تر می شود و باعث می شود که استخوان فک از بین برود.']
-----
['مواد غذایی که باید برای سلامت دهان، لثه و دندان باید بخوریم چگونه برخی غذاها به دندان کمک می کنند و لثه سالم می ماند: دندان ها و لثه ها بخشی از بدن هستند، و به خاطر ارتباطشان با بخش های دیگر

In [ ]:
# argmaxes = np.argpartition(list(h.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')

# argmaxes = np.argpartition(list(a.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')

### <strong><font color=#CF9E50></span> Family Health Docs Analysis </strong>

In [ ]:
vectorizer.fit([' '.join(doc) for doc in all_family_health_tokens_nonsymbol])
X = vectorizer.transform([' '.join(doc) for doc in all_family_health_tokens_nonsymbol])

In [ ]:
P = X.dot(X.T)

In [ ]:
P_norm = normalize(P, norm='l1')
np.sum(P_norm[2,:])

0.9999999999999991

In [ ]:
G = nx.from_numpy_matrix(P_norm.toarray())

In [ ]:
page_rank = nx.pagerank(G, alpha=0.9)
h, a = nx.hits(G)

In [ ]:
argmaxes = np.argpartition(list(page_rank.values()), -5)[-5:]
for i in argmaxes:
  print(all_family_health_doc_sentences[i], end='\n-----\n')

['با این کار، علاوه بر اینکه حجم سبزی ها کم می شود و جای کمتری را هم در فریزر خواهند گرفت، عمل آنزیم بری (بلانچینگ) هم انجام می شود که مانع از تغییر رنگ سبزی ها و تیره شدن آن ها در فرایند انجماد خواهد شد.۴- حتما سبزی ها را در حجم کم و بسته های کوچک (یک وعده ای) منجمد کنید؛ مثلا اگر قرار است سبزی پلو را در ۴ وعده مصرف کنید، حتما آن را در ۴ کیسه بسته بندی یا در ۴ ظرف مختلف قرار دهید.']
-----
['صندل یکی از بهترین کفش های این دوران انتخاب این کفش ها در فصل های گرم از این جهت اهمیت دارد که به خنک ماندن پای شما کمک می کند بنابراین اگر شما از آن دست زنانی باشید که در دوران بارداری با افزایش دمای بدن رو به رو می شوید، می توانید از این بابت آسوده باشید که با پوشیدن صندل بخشی از گرمای هوا را از خود دور می کنید.']
-----
['اگر این داروها را هم علاوه بر مکمل های آهن مصرف می کنید، پیشنهاد می شود که با داروسازتان صحبت کنید که قرص های خود را در زمان های مختلف در طول روز قرار دهید، تا اطمینان حاصل شود که در شرایط مطلوب هریک را دریافت خواهید کرد.']
-----
['متأسفانه یکی از اشتباه های خانم ها این است که س

In [ ]:
# argmaxes = np.argpartition(list(h.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')

# argmaxes = np.argpartition(list(a.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')

### <strong><font color=#CF9E50></span> Mental Health Docs Analysis </strong>

In [ ]:
vectorizer.fit([' '.join(doc) for doc in all_mental_health_tokens_nonsymbol])
X = vectorizer.transform([' '.join(doc) for doc in all_mental_health_tokens_nonsymbol])

In [ ]:
P = X.dot(X.T)

In [ ]:
P_norm = normalize(P, norm='l1')
np.sum(P_norm[2,:])

1.0000000000000022

In [ ]:
G = nx.from_numpy_matrix(P_norm.toarray())

In [ ]:
page_rank = nx.pagerank(G, alpha=0.9)
h, a = nx.hits(G)

In [ ]:
argmaxes = np.argpartition(list(page_rank.values()), -5)[-5:]
for i in argmaxes:
  print(all_mental_health_doc_sentences[i], end='\n-----\n')

['علائم و نشانه های اختلال وسواس فکری و عملی OCD فقط به این دلیل که شما افکار وسواسی یا رفتارهای اجباری دارید، به این معنا نیست که اختلال وسواس فکری داریدبا داشتن OCD، این افکار و رفتارها موجب پریشانی های بزرگی در افراد می شوند، زمان زیادی را به خود اختصاص می دهند (حداقل یک ساعت در روز) و در زندگی روزمره و روابط شما دخالت می کنند اکثر افراد مبتلا به اختلال وسواس فکری، دارای جفت وسواس ها و اجبارها هستند، اما برخی از افراد مبتلا فقط یکی از آنها را تجربه می کنند.']
-----
['بدخلقی و بی حوصلگی، ۳۰ میلیون آمریکایی را درگیر کرده است هفتاد درصد کسانی که نخستین دوره افسردگی را تجربه می کنند، مجددا با آن روبه رو می شوند آمار موجود حاکی از آن است که مردان و زنان هر دو به یک اندازه افسرده می شوند اما زنان بیش از مردان برای درمان خود به پزشک مراجعه می کنند هزینه های درمانی مرتبط با آن فقط در آمریکا به ۱۶ میلیارد دلار در سال می رسد.']
-----
['شاید از مرگ کسی در گذشته، ناراحت هستید شاید با خودتان فکر کنید این که فردی که یک سال پیش یا شش ماه گذشته از دنیا رفته دیگر نباید در زمان کنونی احساسات شما را ب

In [ ]:
# argmaxes = np.argpartition(list(h.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')

# argmaxes = np.argpartition(list(a.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')

### <strong><font color=#CF9E50></span> Hair and Skin Docs Analysis </strong>

In [ ]:
vectorizer.fit([' '.join(doc) for doc in all_hair_skin_tokens_nonsymbol])
X = vectorizer.transform([' '.join(doc) for doc in all_hair_skin_tokens_nonsymbol])

In [ ]:
P = X.dot(X.T)

In [ ]:
P_norm = normalize(P, norm='l1')
np.sum(P_norm[2,:])

0.9999999999999983

In [ ]:
G = nx.from_numpy_matrix(P_norm.toarray())

In [ ]:
page_rank = nx.pagerank(G, alpha=0.9)
h, a = nx.hits(G)

In [ ]:
argmaxes = np.argpartition(list(page_rank.values()), -5)[-5:]
for i in argmaxes:
  print(all_hair_skin_doc_sentences[i], end='\n-----\n')

['مواد لازم: ۱ قاشق غذاخوری روغن آرگان ۲ قاشق غذاخوری جو دو سر ۱ قاشق چای خوری عسل طرز تهیه و طریقه استفاده: ابتدا جو دوسر را در آب گرم بخیسانید تا کاملا نرم شود و سپس روغن و عسل را به آن بیفزایید، به خوبی هم بزنید تا زمانی که ترکیبی یکدست حاصل شود و سپس ماسک به دست آمده را روی صورتتان اعمال کنید و بعد از حدود ۱۰ دقیقه با آب ولرم و سپس آب سرد شستشو دهید.']
-----
['از ماسک به صورت صحیح استفاده کنید مثلا بهره گیری از جعفری، کرفس و لیموترش به شکل ماسک به سلامت پوست کمک می کند اما اگر این ماسک را بزنید و در معرض آفتاب قرار گیرید، در صورت شما لک ایجاد می شود زیرا آب جعفری حاوی ماده ای به نام «سورالن» است که در برابر اشعه مضر آفتاب رنگدانه های پوستی ایجاد می کند.']
-----
['ماسک مو برای کمک به رفع شوره سر مواد لازم: ۱ قاشق غذاخوری روغن آرگان ۱ قاشق غذاخوری روغن نارگیل ۱ قاشق غذاخوری آب لیمو طرز تهیه و طریقه استفاده: تمام مواد را با یکدیگر ترکیب کنید و روی حرارت قرار دهید و به خوبی هم بزنید و اجازه دهید تا کمی خنک شود و سپس روی پوست سر و ساقه موهایتان اعمال کنید و به آرامی به مدت ۱۰ دقیقه ماسا

In [ ]:
# argmaxes = np.argpartition(list(h.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')

# argmaxes = np.argpartition(list(a.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')

### <strong><font color=#CF9E50></span> Prophylaxis and Disease Docs Analysis </strong>

In [ ]:
vectorizer.fit([' '.join(doc) for doc in all_prophylaxis_disease_tokens_nonsymbol])
X = vectorizer.transform([' '.join(doc) for doc in all_prophylaxis_disease_tokens_nonsymbol])

In [ ]:
P = X.dot(X.T)

In [ ]:
P_norm = normalize(P, norm='l1')
np.sum(P_norm[2,:])

0.9999999999999997

In [ ]:
G = nx.from_numpy_matrix(P_norm.toarray())

In [ ]:
page_rank = nx.pagerank(G, alpha=0.9)
h, a = nx.hits(G)

In [ ]:
argmaxes = np.argpartition(list(page_rank.values()), -5)[-5:]
for i in argmaxes:
  print(all_hair_skin_doc_sentences[i], end='\n-----\n')

In [ ]:
# argmaxes = np.argpartition(list(h.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')

# argmaxes = np.argpartition(list(a.values()), -5)[-5:]
# for i in argmaxes:
#   print(all_nutrition_doc_sentences[i], end='\n-----\n')